__Описание источников данных:__

* train.csv - пары "заявка - целевая переменная", для этой выборки нужно собрать признаки и обучить модель;
* test.csv - пары "заявки - прогнозное значение", для этой выборки нужно собрать признаки и построить прогнозы;
* bki.csv - данные БКИ о предыдущих кредитах клиента;
* client_profile.csv - клиентский профиль, некоторые знания, которые есть у компании о клиенте;
* payments.csv - история платежей клиента;
* applications_history.csv - история предыдущих заявок клиента.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*test.shape))

train.shape = 110093 rows, 3 cols
test.shape = 165141 rows, 2 cols


__train.csv__

In [3]:
train.head()

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
0,123687442,0,Cash
1,123597908,1,Cash
2,123526683,0,Cash
3,123710391,1,Cash
4,123590329,1,Cash


Номер заявки уникальный идентификатор в таблице

In [5]:
train["APPLICATION_NUMBER"].nunique()

110093

In [12]:
train["APPLICATION_NUMBER"].min()

123423341

In [13]:
train["APPLICATION_NUMBER"].max()

123730843

In [14]:
123730843-123423341

307502

In [6]:
train["NAME_CONTRACT_TYPE"].nunique()

2

In [7]:
train["NAME_CONTRACT_TYPE"].value_counts()

Cash           99551
Credit Card    10542
Name: NAME_CONTRACT_TYPE, dtype: int64

In [8]:
train["TARGET"].value_counts()

0    101196
1      8897
Name: TARGET, dtype: int64

__test.csv__

In [9]:
test.head()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE
0,123724268,Cash
1,123456549,Cash
2,123428178,Credit Card
3,123619984,Cash
4,123671104,Cash


Номер заявки уникальный идентификатор в таблице

In [10]:
test["APPLICATION_NUMBER"].nunique()

165141

In [15]:
test["APPLICATION_NUMBER"].min()

123423342

In [16]:
test["APPLICATION_NUMBER"].max()

123730851

In [17]:
123730851-123423342

307509

In [11]:
test["NAME_CONTRACT_TYPE"].value_counts()

Cash           149432
Credit Card     15709
Name: NAME_CONTRACT_TYPE, dtype: int64

In [18]:
set(test["APPLICATION_NUMBER"]) & set(train["APPLICATION_NUMBER"])

set()

__bki.csv__

In [21]:
bki = pd.read_csv("bki.csv")
print("bki.shape = {} rows, {} cols".format(*bki.shape))

bki.shape = 945234 rows, 17 cols


In [22]:
bki.head()

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,123538884,5223613,Active,currency 1,718.0,0,377.0,NaN,19386.81,0,675000.00,320265.495,0.0,0.0,Consumer credit,39.0,NaN
1,123436670,6207544,Closed,currency 1,696.0,0,511.0,511.0,0.00,0,93111.66,0.000,0.0,0.0,Consumer credit,505.0,NaN
2,123589020,6326395,Closed,currency 1,165.0,0,149.0,160.0,NaN,0,36000.00,0.000,0.0,0.0,Consumer credit,150.0,0.0
3,123494590,6606618,Active,currency 1,55.0,0,310.0,NaN,NaN,0,38664.00,37858.500,NaN,0.0,Consumer credit,15.0,NaN
4,123446603,5046832,Active,currency 1,358.0,0,35.0,NaN,NaN,0,67500.00,0.000,0.0,0.0,Credit card,116.0,NaN


In [23]:
bki["APPLICATION_NUMBER"].nunique()

273131

In [24]:
bki["APPLICATION_NUMBER"].value_counts()

123444199    63
123493043    51
123641404    50
123603494    39
123604794    36
             ..
123465619     1
123459476     1
123449243     1
123756771     1
123734017     1
Name: APPLICATION_NUMBER, Length: 273131, dtype: int64

In [26]:
bki["CREDIT_ACTIVE"].value_counts()

Closed      594315
Active      347323
Sold          3583
Bad debt        13
Name: CREDIT_ACTIVE, dtype: int64

In [61]:
bki[bki["APPLICATION_NUMBER"]==123604794]

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
2441,123604794,5022625,Closed,currency 1,276.0,0,214.0,234.0,NaN,0,36000.00,0.000,NaN,0.0,Consumer credit,234.0,NaN
25276,123604794,5026176,Closed,currency 1,515.0,0,454.0,486.0,NaN,0,40500.00,0.000,NaN,0.0,Consumer credit,486.0,NaN
26902,123604794,5095767,Closed,currency 1,663.0,0,602.0,640.0,NaN,0,18000.00,0.000,NaN,0.0,Consumer credit,640.0,NaN
62594,123604794,5079117,Active,currency 1,220.0,0,26.0,NaN,0.0,0,67765.50,19903.500,0.0,0.0,Consumer credit,10.0,10505.7
120928,123604794,5050396,Closed,currency 1,914.0,0,853.0,899.0,NaN,0,13500.00,0.000,NaN,0.0,Consumer credit,899.0,0.0
149705,123604794,5018643,Closed,currency 1,234.0,0,173.0,185.0,NaN,0,40500.00,0.000,NaN,0.0,Consumer credit,185.0,0.0
150803,123604794,5066395,Closed,currency 1,821.0,0,762.0,797.0,NaN,0,27000.00,0.000,NaN,0.0,Consumer credit,797.0,0.0
187962,123604794,5026552,Closed,currency 1,30.0,0,NaN,1.0,NaN,0,13500.00,0.000,NaN,0.0,Microloan,1.0,0.0
210264,123604794,5032011,Closed,currency 1,1124.0,0,1063.0,1108.0,NaN,0,4500.00,0.000,NaN,0.0,Consumer credit,1108.0,0.0
219874,123604794,5054762,Closed,currency 1,365.0,0,304.0,346.0,NaN,0,40500.00,0.000,NaN,0.0,Consumer credit,346.0,0.0


Отношения между таблицей train и bki один ко многим, аналогично test и bki

In [33]:
len(set(test["APPLICATION_NUMBER"]) & set(bki["APPLICATION_NUMBER"]))

126469

In [34]:
len(set(train["APPLICATION_NUMBER"]) & set(bki["APPLICATION_NUMBER"]))

84508

Не все заявки из трейна и теста имеют кредитную историю в бюро кредитных историй. Проверим гипотезу о том, что вероятность не выплаты кредита больше у тех заявок, по которым нет кредитной истории

In [53]:
app_without_bki = set(train["APPLICATION_NUMBER"]) - set(bki["APPLICATION_NUMBER"])

In [55]:
print("Число заявок, по которым нет кредитной истории: {}".format(len(app_without_bki)))

Число заявок, по которым нет кредитной истории: 25585


In [60]:
train[train["APPLICATION_NUMBER"].isin(app_without_bki)]["TARGET"].value_counts()

0    23592
1     1993
Name: TARGET, dtype: int64

Вывод: стоит сделать признак "наличие кредитной истории", если у заявки нет кредитной истории вероятность дефолта по ней 22.4%

__client_profile.csv__

In [62]:
client_profile = pd.read_csv("client_profile.csv")
print("client_profile.shape = {} rows, {} cols".format(*client_profile.shape))

client_profile.shape = 250000 rows, 24 cols


In [63]:
client_profile.head()

,APPLICATION_NUMBER,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123666076,F,0,157500.0,270000.0,13500.0,Incomplete higher,Civil marriage,0.008068,8560,...,2.0,0.329471,0.236315,0.678568,0.0,0.0,0.0,0.0,1.0,2.0
1,123423688,F,0,270000.0,536917.5,28467.0,Secondary / secondary special,Married,0.020246,23187,...,2.0,NaN,0.442295,0.802745,0.0,0.0,0.0,0.0,1.0,1.0
2,123501780,M,1,427500.0,239850.0,23850.0,Incomplete higher,Married,0.072508,14387,...,3.0,0.409017,0.738159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,123588799,M,0,112500.0,254700.0,17149.5,Secondary / secondary special,Married,0.019101,14273,...,2.0,NaN,0.308994,0.590233,0.0,0.0,0.0,0.0,0.0,3.0
4,123647485,M,0,130500.0,614574.0,19822.5,Lower secondary,Married,0.022625,22954,...,2.0,NaN,0.739408,0.156640,0.0,0.0,1.0,0.0,0.0,6.0
